In [1]:
import vaex
import vaex.jupyter
import numpy as np
import pylab as plt
df = vaex.example()

In [2]:
selection = df.select(True)
@vaex.jupyter.interactive_selection(df)
def plot(*args, **kwargs):
    df.plot(df.x, df.y, what=np.log(vaex.stat.count()+1), selection=True)
    plt.show()

Output()

In [3]:
df.select(df.x > 0)

In [4]:
def display_radio():
    import ipywidgets as widgets
    radio = widgets.RadioButtons(
        options=['x > 2', 'y < 2', 'x > 2 & y < 2'],
        disabled=False
    )
    def handle(change):
        if change['name'] == 'index':
            if change.new==0:
                df.select(df.x > 2)
            elif change.new==1:
                df.select(df.y < 2)
            elif change.new==2:
                df.select((df.x > 2) & (df.y < 2))

    radio.observe(handle)
    display(radio)

In [5]:
display_radio()

RadioButtons(options=('x > 2', 'y < 2', 'x > 2 & y < 2'), value='x > 2')

In [6]:
#df.plot_widget(df.x, df.y, [])
#df.plot(df.x, df.y, f='log1p',figsize=(40,30))
#df.plot(df.x, df.y)

In [7]:
### Imports for the navigation
from ipywidgets import Button, HBox, VBox,Layout, FloatSlider, interact, interactive
import ipywidgets as widgets

In [8]:
### Added very simple navigation
def zoom_fun_gen(t):
    x, y = t
    def zoom_fun(b):
        global xlim_l
        global xlim_r
        global ylim_d
        global ylim_u
        x_d = xlim_r - xlim_l
        y_d = ylim_u - ylim_d
        x_diff = x_d/3
        y_diff = y_d/3
        xlim_l+=x*x_diff # 0, 1, 2
        xlim_r-=(2-x)*x_diff # 2, 1, 0
        ylim_d+=y*y_diff # 0, 1, 2
        ylim_u-=(2-y)*y_diff # 2, 1, 0
        df.select(df.x > 0)
    return zoom_fun

In [36]:
xlim_l = -28
xlim_r = 28
ylim_d = -28
ylim_u = 28
x_c = 0
y_c = 0
dirs = ['zoom top left', 'zoom left', 'zoom bottom left', 
         'zoom top middle', 'zoom middle', 'zoom bottom middle',
         'zoom top right',  'zoom right',  'zoom bottom right']
dirs_n = [(0, 2), (0, 1), (0, 0), (1, 2), (1, 1), (1, 0), (2, 2), (2, 1), (2, 0)]
buttons = [Button(description=d) for d in dirs]
for i, b in enumerate(buttons):
    b.on_click(zoom_fun_gen(dirs_n[i]))

In [10]:
def reset(change):
    global xlim_l
    global xlim_r
    global ylim_d
    global ylim_u
    xlim_l=-28
    xlim_r=28
    ylim_d = -28
    ylim_u = 28
    df.select(df.x > 0)
resetb = Button(description="reset")
resetb.on_click(reset)
percent_shift = .2
def left_move(change):
    global xlim_l
    global xlim_r
    global percent_shift
    x_d = xlim_r - xlim_l
    xlim_l-=percent_shift*x_d
    xlim_r-=percent_shift*x_d
    df.select(df.x > 0)
    
def right_move(change):
    global xlim_l
    global xlim_r
    global percent_shift
    x_d = xlim_r - xlim_l
    xlim_l+=percent_shift*x_d
    xlim_r+=percent_shift*x_d
    df.select(df.x > 0)
    
def up_move(change):
    global ylim_d
    global ylim_u
    global percent_shift
    y_d = ylim_u - ylim_d
    ylim_d+=percent_shift*y_d
    ylim_u+=percent_shift*y_d
    df.select(df.x > 0)
    
def down_move(change):
    global ylim_d
    global ylim_u
    global percent_shift
    y_d = ylim_u - ylim_d
    ylim_d-=percent_shift*y_d
    ylim_u-=percent_shift*y_d
    df.select(df.x > 0)

rightb = Button(description="right")
rightb.on_click(right_move)
leftb = Button(description="left")
leftb.on_click(left_move)
upb = Button(description="up")
upb.on_click(up_move)
downb = Button(description="down")
downb.on_click(down_move)

In [11]:
left_col   = VBox([button for button in buttons[:3]])
middle_col = VBox([button for button in buttons[3:6]])
right_col  = VBox([button for button in buttons[6:]])

display(HBox([left_col, middle_col, right_col]))
display(resetb)
display(HBox([leftb, rightb, VBox([upb, downb])]))

Button(description='reset', style=ButtonStyle())

In [12]:
selection = df.select(True)
@vaex.jupyter.interactive_selection(df)
def plot(*args, **kwargs):
    df.plot(df.x, df.y, what=np.log(vaex.stat.count()+1), limits = [[xlim_l,xlim_r],[ylim_d,ylim_u]], selection=False)
    #plt.xlim(xlim_l, xlim_r)
    #plt.ylim(ylim_d, ylim_u)
    plt.show()

Output()

In [13]:
df.select(df.x)

In [21]:
left_col   = VBox([button for button in buttons[:3]])
middle_col = VBox([button for button in buttons[3:6]])
right_col  = VBox([button for button in buttons[6:]])

display(HBox([left_col, middle_col, right_col]))
display(resetb)
display(HBox([leftb, rightb, VBox([upb, downb])]))

Button(description='reset', style=ButtonStyle())

In [37]:
def zoom_in(b):
    global xlim_l
    global xlim_r
    global ylim_d
    global ylim_u
    global x_c
    global y_c
    #x_c = (xlim_r + xlim_l)/2
    #y_c = (ylim_u + ylim_d)/2
    x_rad = xlim_r - x_c
    y_rad = ylim_u - y_c
    x_rad_new = x_rad / zoom_size
    y_rad_new = x_rad / zoom_size
    xlim_l = x_c - x_rad_new
    xlim_r = x_c + x_rad_new
    ylim_d = y_c - y_rad_new
    ylim_u = y_c + y_rad_new
    df.select(df.x > 0)
def zoom_out(b):
    global xlim_l
    global xlim_r
    global ylim_d
    global ylim_u
    global x_c
    global y_c
    #x_c = (xlim_r + xlim_l)/2
    #y_c = (ylim_u + ylim_d)/2
    x_rad = xlim_r - x_c
    y_rad = ylim_u - y_c
    x_rad_new = x_rad * zoom_size
    y_rad_new = x_rad * zoom_size
    xlim_l = x_c - x_rad_new
    xlim_r = x_c + x_rad_new
    ylim_d = y_c - y_rad_new
    ylim_u = y_c + y_rad_new
    df.select(df.x > 0)

buttonReset = Button(description="reset",layout=Layout(width="50px"))
buttonReset.on_click(reset)
buttonZoomOut = Button(description='-',layout=Layout(width="50px"))
buttonZoomOut.on_click(zoom_out);
buttonZoomIn = Button(description='+',layout=Layout(width="50px"))
buttonZoomIn.on_click(zoom_in);
buttonPanUp = Button(description=u'\u2191', layout=Layout(width="50px"))
buttonPanDown = Button(description=u'\u2193', layout=Layout(width="50px"))
buttonPanRight = Button(description=u'\u2192', layout=Layout(width="50px"))
buttonPanLeft = Button(description=u'\u2190', layout=Layout(width="50px"))
buttonPanRight.on_click(right_move)
buttonPanLeft.on_click(left_move)
buttonPanUp.on_click(up_move)
buttonPanDown.on_click(down_move)

def updateZoomSize(b):
    global zoom_size
    zoom_size = b;
def updatePanSize(b):
    global percent_shift
    percent_shift = b;
inputPanSize = interactive(updatePanSize, b=FloatSlider(value=0.2, min=0.05, max=0.5,step=0.05,description='Pan %'));
inputZoomSize = interactive(updateZoomSize, b=FloatSlider(value=1.2, min=1.2, max=4.0,step=0.2,description='Zoom %'));

nav_buttons = HBox([buttonPanLeft,buttonPanUp,buttonPanDown,buttonPanRight,buttonZoomIn,buttonZoomOut,buttonReset])
nav_input = HBox([inputZoomSize, inputPanSize])
nav_controls = VBox([nav_buttons, nav_input])
#display(nav_controls)


In [42]:
@vaex.jupyter.interactive_selection(df)
def plot(*args, **kwargs):
    global a
    #df.plot(df.x, df.y, what=np.log(vaex.stat.count()+1), limits = [[xlim_l,xlim_r],[ylim_d,ylim_u]], selection=False)
    a = df.plot(df.x, df.y, limits = [[xlim_l,xlim_r],[ylim_d,ylim_u]], selection=True)
    plt.show()
    display(nav_controls)

Output()

In [35]:
plt.show()